In [1]:
from matchms.importing import load_from_msp

In [2]:
spectra = list(load_from_msp("./RECETOX_GC-EI_MS_20201028.msp"))

In [3]:
all_spectra = [x for x in spectra]
spectra = all_spectra

In [4]:
for spec in spectra:
    print(spec.metadata_dict())

{'scannumber': '-1', 'ionmode': 'positive', 'spectrumtype': 'Centroid', 'formula': 'C20H12', 'inchikey': 'CSHWQDPOILHKBI-AQZSQYOVSA-N', 'smiles': '[2H]C1=C(C2=C3C(=C1[2H])C4=C(C(=C(C5=C4C(=C(C(=C5[2H])[2H])[2H])C3=C(C(=C2[2H])[2H])[2H])[2H])[2H])[2H])[2H]', 'authors': 'Price et al., RECETOX, Masaryk University (CZ)', 'instrument': 'Q Exactive GC Orbitrap GC-MS/MS', 'ionization': 'EI+', 'license': 'CC BY-NC', 'num_peaks': '33', 'compound_name': 'Perylene_2H12', 'retention_time': None, 'retention_index': 2876.0, 'precursor_mz': 264.16858, 'adduct': '[M]+', 'collision_energy': '70eV', 'instrument_type': 'GC-EI-Orbitrap'}
{'scannumber': '-1', 'ionmode': 'positive', 'spectrumtype': 'Centroid', 'formula': 'C20H12', 'inchikey': 'CSHWQDPOILHKBI-UHFFFAOYSA-N', 'smiles': 'C1=CC2=C3C(=C1)C1=CC=CC4=C1C(=CC=C4)C3=CC=C2', 'authors': 'Price et al., RECETOX, Masaryk University (CZ)', 'instrument': 'Q Exactive GC Orbitrap GC-MS/MS', 'ionization': 'EI+', 'license': 'CC BY-NC', 'peak_comments': {113.0385

In [5]:
import numpy as np
from collections import Counter

def extract_deltas(spectra):
    all_deltas = []
    for spectrum in spectra:
        deltas = spectrum.mz[:, np.newaxis] - spectrum.mz
        all_deltas.extend(deltas[~np.eye(len(spectrum.mz), dtype=bool)])
    return all_deltas

def count_deltas(deltas):
    return Counter(deltas)

In [ ]:
# Compute the delta histogram
delta_histogram = count_deltas(extract_deltas(spectra[:100]))

# Display the histogram as a bar plot
import matplotlib.pyplot as plt
deltas, counts = zip(*sorted(delta_histogram.items()))
plt.bar(deltas, counts, width=0.1)
plt.xlabel('Delta values')
plt.ylabel('Frequency')
plt.title('Histogram of Delta Values')
plt.show()

In [6]:
import intervaltree
import tqdm

delta_tree = intervaltree.IntervalTree()

for delta in tqdm.tqdm(extract_deltas(spectra[:100])):
    delta = round(delta, 4)
    matches = delta_tree.at(delta)
    if matches:
        for match in matches:
            match.data["sum"] += delta
            match.data["measurements"] += 1
            match.data["mean"] = match.data['sum'] / match.data['measurements']
    else:
        mz_err = abs(delta / 1e6 * 5)
        delta_tree.addi(delta - mz_err - 1e-4, delta + mz_err + 1e-4, {"sum": delta, "measurements": 1, "mean": delta})


100%|██████████| 416752/416752 [00:03<00:00, 107474.85it/s]


In [7]:
print(len(delta_tree))
for x in sorted(delta_tree, key=lambda x: -x.data['measurements'])[:50]:
    print(x)

103558
Interval(-1.0080050395, -1.0077949605, {'sum': -985.6658999999833, 'measurements': 978, 'mean': -1.0078383435582652})
Interval(1.0077949605, 1.0080050395, {'sum': 985.6658999999833, 'measurements': 978, 'mean': 1.0078383435582652})
Interval(1.9969900145000001, 1.9972099855, {'sum': 1695.4918000000187, 'measurements': 849, 'mean': 1.9970457008245215})
Interval(-1.9972099855, -1.9969900145000001, {'sum': -1695.4918000000187, 'measurements': 849, 'mean': -1.9970457008245215})
Interval(-12.00016, -11.99984, {'sum': -9671.995799999992, 'measurements': 806, 'mean': -11.999994789081875})
Interval(11.99984, 12.00016, {'sum': 9671.995799999992, 'measurements': 806, 'mean': 11.999994789081875})
Interval(-1.0078050385000001, -1.0075949615, {'sum': -769.9450999999898, 'measurements': 764, 'mean': -1.0077815445026044})
Interval(1.0075949615, 1.0078050385000001, {'sum': 769.9450999999898, 'measurements': 764, 'mean': 1.0077815445026044})
Interval(34.96862515549999, 34.9691748445, {'sum': 2580

In [17]:
import heapq
from joblib import Parallel, delayed
from MassDeltaExplainer import MassDeltaExplainer
from itertools import product

DEBUG = True

def default_basis():
    import json
    data = json.load(open("/Users/mitchjo/khipu-1/notebooks/default_basis.json"))
    iso_basis, iso_basis_annots = [x[0] for x in data['iso_basis']], [x[1] for x in data['iso_basis']]
    frag_basis, frag_basis_annots = [x[0] for x in data['frag_basis']], [x[1] for x in data['frag_basis']]
    return iso_basis, frag_basis, iso_basis_annots, frag_basis_annots

def compute_chunk(combos, combined_basis):
    """
    Compute net masses for a chunk of coefficient combinations.
    """
    linear_combos = np.array(combos)
    net_masses = np.dot(linear_combos, combined_basis)
    return net_masses

def expand_basis_parallel(iso_basis, frag_basis, max_iso, max_frag, iso_basis_annots, frag_basis_annots, n_jobs=-1):
    explained_masses = intervaltree.IntervalTree()
    combined_basis = np.array(iso_basis + frag_basis)
    max_ranges = [max_iso] * len(iso_basis) + [max_frag] * len(frag_basis)
    combo_basis = [np.arange(-x, x + 1) for x in max_ranges]
    
    # Divide Cartesian product into chunks for parallel processing
    all_combinations = product(*combo_basis)  # Generator for all combinations
    chunk_size = 10_000  # Adjust based on memory and performance needs
    chunks = []
    current_chunk = []
    for combo in all_combinations:
        current_chunk.append(combo)
        if len(current_chunk) >= chunk_size:
            chunks.append(current_chunk)
            current_chunk = []
    if current_chunk:
        chunks.append(current_chunk)

    # Parallel computation of net masses
    results = Parallel(n_jobs=8)(delayed(compute_chunk)(chunk, combined_basis) for chunk in chunks)

    # Flatten results and build IntervalTree
    net_masses = np.concatenate(results)
    tolerance = abs(net_masses / 1e6 * 5) + 1e-4
    seen_masses = set()
    for net_mass, tol in tqdm.tqdm(zip(net_masses, tolerance), total=len(net_masses)):
        if net_mass not in seen_masses:
            explained_masses.addi(net_mass - tol, net_mass + tol)
            seen_masses.add(net_mass)

    return explained_masses



def expand_basis(iso_basis, frag_basis, max_iso, max_frag, iso_basis_annots, frag_basis_annots):
    explained_masses = intervaltree.IntervalTree()

    # Combine bases
    combined_basis = np.array(iso_basis + frag_basis)
    max_ranges = [max_iso] * len(iso_basis) + [max_frag] * len(frag_basis)

    # Generate all coefficient combinations
    combo_basis = [np.arange(-x, x + 1) for x in max_ranges]
    all_combinations = list(product(*combo_basis))  # Cartesian product of ranges

    # Compute net masses
    linear_combos = np.array(all_combinations)
    net_masses = np.dot(linear_combos, combined_basis)

    # Batch update the interval tree
    tolerance = abs(net_masses / 1e6 * 5) + 1e-4
    seen_masses = set()
    for net_mass, tol in tqdm.tqdm(zip(net_masses, tolerance), total=len(net_masses)):
        if net_mass not in seen_masses:
            explained_masses.addi(net_mass - tol, net_mass + tol)
            seen_masses.add(net_mass)

    if DEBUG:
        print("\tBasis: ", len(combined_basis))
        for b, ba in zip(combined_basis, iso_basis_annots + frag_basis_annots):
            print("\t\t", b, ba)
        #print("\tExpanded: ", len(explained_masses))
        #for x in explained_masses:
        #    print("\t", x)
    return explained_masses

def score_basis(delta_tree, explained_masses, print_score=False):
    explained, unexplained = 0, 0
    for x in delta_tree:
        mean_mz, count_mz = x.data['mean'], x.data['measurements']
        if explained_masses.at(mean_mz):
            explained += count_mz
        else:
            unexplained += count_mz
    if DEBUG or print_score:
        print("\tSCORE: ", explained, unexplained, " Explains: ", (100* explained / (unexplained + explained)), "%")
    return explained, unexplained

def grow_basis(heap, iso_basis, frag_basis, iso_basis_annots, frag_basis_annots, mde_instance):
    while True:
        (count, mass) = heapq.heappop(heap)
        #if DEBUG:
        #    print("\tNext Mass: ", mass, "SKIP" if bool(explained_masses.at(mass)) else "")
        if not explained_masses.at(mass):
            deconv = mde.explains(mass)
            if deconv['explained']:
                for solution in deconv['solutions']:
                    if solution['isotope_only']:
                        iso_basis_annots.append([solution['key'] + " " + str(count)])
                        iso_basis.append(solution['solution_mass_delta'])
                        print("ADDING iso: ", solution['solution_mass_delta'], solution['key'])

                    else:
                        frag_basis_annots.append([solution['key'] + " " + str(count)])
                        frag_basis.append(solution['solution_mass_delta'])
                        print("ADDING frag: ", solution['solution_mass_delta'], solution['key'])
                break
    return heap, iso_basis, frag_basis, iso_basis_annots, frag_basis_annots

def heapify_dtree(delta_tree):
    heap = []
    heapq.heapify(heap)
    for x in delta_tree:
        heapq.heappush(heap, (-x.data['measurements'], x.data['mean']))
    return heap

MAX_ISO = 1
MAX_FRAG = 3
MAX_ITER = 10

mde = MassDeltaExplainer("./isotope_components.csv", MIN_ISO_PROB=0.02)
heap = heapify_dtree(delta_tree)
iso_basis, frag_basis, iso_basis_annots, frag_basis_annots = default_basis()

for num_iter in range(MAX_ITER):
    print("Iter: ", num_iter, " / ", MAX_ITER, " basis: ", len(iso_basis + frag_basis))
    explained_masses = expand_basis_parallel(iso_basis, frag_basis, MAX_ISO, MAX_FRAG, iso_basis_annots, frag_basis_annots)
    explained, unexplained = score_basis(delta_tree, explained_masses, print_score=True)
    if explained / (explained + unexplained) > .90:
        break
    else:
        heap, iso_basis, frag_basis, iso_basis_annots, frag_basis_annots = grow_basis(heap, iso_basis, frag_basis, iso_basis_annots, frag_basis_annots, mde)
print("SOLUTION: ")
print("Isotopologues: ")
for ib, iba in zip(iso_basis, iso_basis_annots):
    print("\t", ib, iba)
print("Fragments: ")
for fb, fba in zip(frag_basis, frag_basis_annots):
    print("\t", fb, fba)



enumerating depth 2: 100%|██████████| 573/573 [00:00<00:00, 3707.32it/s]


Iter:  0  /  10  basis:  8


 51%|█████     | 537066/1058841 [00:06<00:06, 74792.87it/s]